In [44]:
import os
import json
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from subprocess import Popen

## variables
modO:str = "F189-A"
subseqs:list = []
VAZY_2:dict = {}

## dir path"
data_dir:str = "./../../../../testymolo/media/data/"
Vazy2_dir:str = os.path.join(data_dir, "transfert/")

## read data files
""" >> "trace_modO.json"  """
trace_modO:list = []
with open(os.path.join(Vazy2_dir,'trace_modO.json')) as file:
    trace_modO = json.load(file)
with open(os.path.join(data_dir, "Organism.temp.json")) as handle:
    VAZY_2['Organism'] = json.load(handle)
with open(os.path.join(data_dir, "Protein.temp.json")) as handle:
    VAZY_2['Protein'] = json.load(handle)
with open(os.path.join(data_dir, "Subseq.temp.json")) as handle:
    VAZY_2['subseq'] = json.load(handle)


In [45]:
def get_organism(DB_ac:str) -> str:
    for item in VAZY_2['Protein']:
        if DB_ac == str(item['data_ac']):
            return str(item['organism'])
    return ''

def get_origin(DB_ac:str) -> dict:
    for item in VAZY_2['Protein']:
        if DB_ac == str(item['data_ac']) and not item['derivedFromPP']:
            return {'header':item['header'], 'sequence':item['sequence']}
    return None

def get_fasta(*sseq) -> dict:
    origin = get_origin(sseq[0])
    if origin:
        start:int = int(sseq[2])
        end:int = int(sseq[3])
        header:str = f">{origin['header']}:[{start}-{end}]"
        sequence:str = origin['sequence'][start-1:end-1]
        return {'header':header, 'sequence':sequence} 
    return None

In [46]:
for sseq in trace_modO[modO]:
    fasta = get_fasta(*sseq)
    if fasta is not None:
        subseqs.append(fasta)
print(*subseqs, sep='\n')

{'header': '>ORF1ab_polyprotein__(@473):[3929-4868]', 'sequence': 'SVAGASDFDKNYLNRVRGSSEARLIPLASGCDPDVVKRAFDVCNKESAGMFQNLKRNCARFQELRDTEDGNLEYLDSYFVVKQTTPSNYEHEKSCYEDLKSEVTADHDFFVFNKNIYNISRQRLTKYTMMDFCYALRHFDPKDCEVLKEILVTYGCIEDYHPKWFEENKDWYDPIENSKYYVMLAKMGPIVRRALLNAIEFGNLMVEKGYVGVITLDNQDLNGKFYDFGDFQKTAPGAGVPVFDTYYSYMMPIIAMTDALAPERYFEYDVHKGYKSYDLLKYDYTEEKQELFQKYFKYWDQEYHPNCRDCSDDRCLIHCANFNILFSTLIPQTSFGNLCRKVFVDGVPFIATCGYHSKELGVIMNQDNTMSFSKMGLSQLMQFVGDPALLVGTSNNLVDLRTSCFSVCALTSGITHQTVKPGHFNKDFYDFAEKAGMFKEGSSIPLKHFFYPQTGNAAINDYDYYRYNRPTMFDICQLLFCLEVTSKYFECYEGGCIPASQVVVNNLDKSAGYPFNKFGKARLYYEMSLEEQDQLFEITKKNVLPTITQMNLKYAISAKNRARTVAGVSILSTMTNRQFHQKILKSIVNTRNASVVIGTTKFYGGWDNMLRNLIQGVEDPILMGWDYPKCDRAMPNLLRIAASLVLARKHTNCCSWSERIYRLYNECAQVLSETVLATGGIYVKPGGTSSGDATTAYANSVFNIIQATSANVARLLSVITRDIVYDNIKSLQYELYQQVYRRVNFDPAFVEKFYSYLCKNFSLMILSDDGVVCYNNTLAKQGLVADISGFREVLYYQNNVFMADSKCWVEPDLEKGPHEFCSQHTMLVEVDGEPKYLPYPDPSRILGACVFVDDVDKTEPVAVMERYIALAIDAYPLVHHENEEYKKVFFVLLAYIRKLYQELSQNMLMDYSFVMDIDKGSKFWEQEFYENMY

In [47]:
### into fasta
with open("./infile.fasta",'w') as handle:
    for fasta in subseqs:
        handle.write(fasta['header']+'\n')
        handle.write(fasta['sequence']+'\n')


In [48]:
### align
filepath = "./infile.fasta"
out_filepath = "./outfile.fasta"
command:str = str(MuscleCommandline(cmd='muscle', input=filepath, out=out_filepath))
print(command+" -quiet" )  # adding "-quiet" option ... doen't work in v3.8 ?! (but does in v5.)
Popen(command.split(' '))  # command must be a list of words


muscle -in ./infile.fasta -out ./outfile.fasta -quiet



MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

infile 6 seqs, lengths min 926, max 939, avg 929
00:00:00    16 MB(-2%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00    16 MB(-2%)  Iter   1  100.00%  K-mer dist pass 2


<Popen: returncode: None args: ['muscle', '-in', './infile.fasta', '-out', '...>

In [ ]:
class Profile:
    
    def __init__(self, filepath:str) -> None:
        msa:list = []
        with open(filepath) as handle:
            for record in SeqIO.parse(handle, format='fasta'):
                msa.append({'header': record.description, 'sequence':str(record.seq)})
        self.N:int = len(msa) #number of sequences
        self.L:int = len(msa[0]["sequence"]) #length of alignment
        self.ruler:list = []
        for l in range(self.L):
            l += 1
            if(l%10 == 0):
                self.ruler.append({'i':l, 'v':'|'})
            elif(l%5 == 0):
                self.ruler.append({'i':l, 'v':'-'})
            else:
                self.ruler.append({'i':l, 'v':'_'})
        
        # MSA
        self.icons:list = []
        self.origin:list = []
        self.headers:list = []
        self.profile:list = []
        for record in msa:
            subseq_id:str = record['header']
            subseq_id = subseq_id.split('@')[-1].split('.')[-1].split(')')[0]
            sequence:list = []
            p = 0  # real position
            for n in range(1, len(record['sequence'])+1):  # aligned position
                residue = record['sequence'][n]
                if residue != '-':  # is not gap
                    p += 1
                sequence.append({'residue':residue, 'realPos':p, 'alignedPos':n})
            
            self.icons.append({'id':subseq_id})
            self.origin.append({'origin':'null'})
            self.headers.append({'id':subseq_id, 'header':record['header']})
            self.profile.append({'id':subseq_id, 'sequence':sequence})

    def Display(self):
        return {'ruler': self.ruler, 'icons':self.icons, 'origin':self.origin, 'headers':self.headers, 'profile':self.profile}
        
        

In [60]:
def generate_mainfigure_profile(data:dict):
    msa:list = []   
    with open(data["outfile_path"]) as handle:
        for record in SeqIO.parse(handle, format="fasta"):
            msa.append({"header": record.description, "sequence":str(record.seq)})
    # parameters 
    N:int = len(msa) #number of sequences
    L:int = len(msa[0]["sequence"]) #length of alignment
    ### html
    html:str = f"<div class='msa'>"
    ### 
    # div functional button #
    div_icons:str = "<div class='icons' ><span class='whitespace'>_</span><span class='whitespace'>_</span>"
    # div header #
    div_header:str = "<div class='headers' >"
    div_header += "<span class='residue' >Records:</span><span class='whitespace'>_</span>"
    # div graduated ruler #
    # div sequences #
    div_sequences:str = "<div >"
    # visible positions
    div_sequences += f"<div class='scroll_values' ><span id='start' >1</span><span id='middle' ><span class='whitespace'>_</span></span><span id='end' >{L}</span></div>"
    div_sequences += "<div class='sequences' >"
    ruler:str = ""
    for l in range(L):
        l += 1
        #title = position => read as caption on mouseover
        if(l%10 == 0):
            ruler += f"<span title={l} >|</span>"
        elif(l%5 == 0):
            ruler += f"<span title={l} >-</span>"
        else :
            ruler += f"<span class='whitespace' title={l} >_</span>"
    div_sequences += f"<span class='sequence' >{ruler}</span>"
    # div horizontal scroll # 

    for record in msa:
        subseq_id:str = record['header']
        subseq_id = subseq_id.split('@')[-1].split('.')[-1].split(')')[0]
        div_icons += f"<span class='sequence_selected_icon' subseq='{subseq_id}' ><span class='whitespace'>_</span></span>"
        div_header += f"<span class='header' subseq='{subseq_id}' >{record['header']}</span>"
        div_sequences += f"<span class='sequence' subseq='{subseq_id}' >"
        n = 1 #aligned position => read as caption when mouseovered 
        p = 0 #real position
        for residue in record['sequence']:
            if(residue != '-'): #if not a gap
                p += 1
            div_sequences +=  f"<span class='residue' data-residue='{residue}' realpos='{p}' alignedpos='{n}' title='{p}-{n}' >{residue}</span>"
            n += 1
        div_sequences += "</span>"

    div_icons += "</div>"
    div_header += "</div>"
    div_sequences += "</div></div>"

    ### 
    html += div_icons
    html += div_header
    html += div_sequences
    ### 
    html += "</div>"
    return html 

In [62]:
generate_mainfigure_profile({'outfile_path':"./outfile.fasta"})

"<div class='msa'><div class='icons' ><span class='whitespace'>_</span><span class='whitespace'>_</span><span class='sequence_selected_icon' subseq='451' ><span class='whitespace'>_</span></span><span class='sequence_selected_icon' subseq='465' ><span class='whitespace'>_</span></span><span class='sequence_selected_icon' subseq='459' ><span class='whitespace'>_</span></span><span class='sequence_selected_icon' subseq='473' ><span class='whitespace'>_</span></span><span class='sequence_selected_icon' subseq='NP_045299' ><span class='whitespace'>_</span></span><span class='sequence_selected_icon' subseq='375' ><span class='whitespace'>_</span></span></div><div class='headers' ><span class='residue' >Records:</span><span class='whitespace'>_</span><span class='header' subseq='451' >ORF1ab_polyprotein__(@451):[3999-4928]</span><span class='header' subseq='465' >ORF1ab_polyprotein__(@465):[4069-4995]</span><span class='header' subseq='459' >polyprotein__(@459):[4101-5027]</span><span class=

In [61]:
from IPython.core.display import display, HTML
display(HTML(generate_mainfigure_profile({'outfile_path':"./outfile.fasta"})))